# Bangalore Real Estate Costs

We also need to consider the relative costs of leasing or purchasing real estate in a given location. We will use web scraping to get this information and give us an idea of the relative costs of different localities in the city.

In [1]:
import math
import os

import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import plotly.express as px
import requests
from bs4 import BeautifulSoup
from diskcache import Cache
from geopy.distance import distance
from geopy.geocoders import Here
from matplotlib.cm import get_cmap
from matplotlib.colors import rgb2hex
from ratelimiter import RateLimiter
from sklearn.neighbors import BallTree
from tqdm.notebook import tqdm

## Web Scraping
First, we need real estate costs by area/locality in bangalore to use as a reference for our study. We will use the data from the 99acres website for this - usign BeatifulSoup to parse the relevant data to a dataframe.

**Data Source:** [99acres website](https://www.99acres.com/property-rates-and-price-trends-in-bangalore)

In [2]:
url = 'https://www.99acres.com/property-rates-and-price-trends-in-bangalore'
html = requests.get(url)
soup = BeautifulSoup(html.text)

In [3]:
locations = []
prices = []

for row in soup.find('tbody').find_all('tr'):
    if len(row.find_all('th')):
        continue
    else:
        location = row.find_all('td')[0].text
        price = row.find_all('td')[1].text
        locations.append(location)
        prices.append(price)

df_rent = pd.DataFrame({
    'location': locations,
    'price': prices,
})
df_rent.head()

,location,price
0,2nd Stage Nagarbhavi,"Rs. 4,250 - 4,632/sq. ft."
1,Basaveshwara Nagar,-
2,BEML Layout,"Rs. 4,505 - 4,675/sq. ft."
3,Binny Pete,"Rs. 8,415 - 10,582/sq. ft."
4,Chandra Layout,-


In [4]:
# Clean and average prices
def avgPrice(string):
    if string.strip() == '-':
        return np.nan
    else:
        prices = string.strip('Rs.')\
            .strip('/sq. ft.')\
            .replace(',','')\
            .split(' - ')
        prices = map(int, prices)
        average = sum(prices)/2
        return average

In [5]:
df_rent['price'] = df_rent['price'].map(avgPrice)

df_rent = df_rent.dropna().reset_index(drop = True)

print(df_rent.shape)
df_rent.head()

(269, 2)


,location,price
0,2nd Stage Nagarbhavi,4441.0
1,BEML Layout,4590.0
2,Binny Pete,9498.5
3,Dasanapura,4483.5
4,Gopalapura,12516.0


## Reverse Geocoding
This price information is only useful to us if we add geospatial information by converting the neighbourhood names in the original list to geographical coordinates. We have used the HERE API to geocode the data in this case.

In [6]:
# Get HERE API Key from environment variables
HERE_API_KEY = os.environ.get('HERE_API_KEY')
if not HERE_API_KEY:
    raise Exception('Could not find environment variables HERE_API_KEY.')

In [7]:
# Set cache location for HERE requests
here_cache = '../data/cache/here'

# Create HERE geocoder object
geocoder = Here(apikey = HERE_API_KEY)

# Create RateLimiter object
limiter = RateLimiter(max_calls = 20, period = 1)

In [8]:
# Empty lists to store responses
locations = []

queries = df_rent[['location']].to_records(index = True)

new_queries = 0 # Added variable to track number of records retrieved from cache

with Cache(here_cache) as cache:
    for index, address in tqdm(queries):
        query = address + ',Bangalore'
        key = str(query) #! key must be a unique string
        
        if key in cache:
            response = cache[key] # Read cached value
        else:
            with limiter:
                response = geocoder.geocode(query, timeout = 10)
                cache[key] = response # Set cache value
                new_queries += 1
        
        if response:
            location = (index, response.latitude, response.longitude)
            locations.append(location)

print('{} new queries made.'.format(new_queries))
print('{} venues successfully geocoded.'.format(len(locations)))

  0%|          | 0/269 [00:00<?, ?it/s]

0 new queries made.
252 venues successfully geocoded.


In [9]:
df_geocoded = pd.DataFrame(locations, columns=['index','lat','lon'])\
    .set_index('index', drop = True)

# Merge coordinates with rent table
df_rent = df_rent.merge(df_geocoded, how = 'inner', left_index = True, right_index = True)\
    .reset_index(drop = True)

print(df_rent.shape)
df_rent.head()

(252, 4)


,location,price,lat,lon
0,2nd Stage Nagarbhavi,4441.0,12.97784,77.51025
1,BEML Layout,4590.0,12.91556,77.52629
2,Binny Pete,9498.5,12.96563,77.55496
3,Dasanapura,4483.5,13.07627,77.43472
4,Gopalapura,12516.0,13.14077,77.42679


These points can now be plotted on a map to show the distribution of our datapoints. We will also color code the markers to give us a rough idea of the prices at different locations.

In [10]:
# Select matplotlib colormap
cmap = get_cmap('YlGn')

# Scale the price (zero to one) for use with colormap
scaled_price = [price/max(df_rent['price']) for price in df_rent['price'].values]

In [11]:
# Select matplotlib colormap
cmap = get_cmap('winter')

map_centre = (12.9792,77.5916)

m = folium.Map(location = map_centre, zoom_start = 11)

for loc, price, lat, lon in df_rent.to_records(index = False):
    # Scale the price (zero to one) for use with colormap
    scaled_price = price/max(df_rent['price'])
    clr = rgb2hex(cmap(scaled_price))
    
    # Plot CircleMarkers
    folium.CircleMarker(
        location = (lat, lon),
        radius = 5,
        color = clr,
        opacity = 1.0,
        fill = True,
        fill_color = clr,
        fill_opacity = 0.7,
        popup = loc + ': ' + str(price),
    ).add_to(m)
    
m # Display map

## Interpolation

We have a good coverage of price points at different locations to use as a reference. Now, we need to use these values to estimate the real estate prices in each hexagon in our grid - meaning we have to use interpolation to estimate values outside our sample set.

For this project, we will use the Shepard's method of Inverse Distance Weighting, as described [here](https://en.wikipedia.org/wiki/Inverse_distance_weighting).

In [12]:
# Import hexagon addresses
df_hex = gpd.read_feather('../data/bangalore_hex_addresses.feather')
print(df_hex.shape)
df_hex.head()

(942, 10)


,id,hex_id,ward_no,centre_lat,centre_lon,resolution,pop_total,ward_name,geometry,address
0,8861892db3fffff,8861892db3fffff,1,13.113109,77.609520,8,1413.220043,Kempegowda Ward,"POLYGON ((77.61369 13.11060, 77.61378 13.11559...","Yelahanka, Kempegowda, Yelahanka Zone, Bengalu..."
1,886016975dfffff,886016975dfffff,1,13.128071,77.609773,8,1762.379434,Kempegowda Ward,"POLYGON ((77.61395 13.12557, 77.61403 13.13055...","Kempegowda, Yelahanka Zone, Bengaluru, Bangalo..."
2,8860169759fffff,8860169759fffff,1,13.120601,77.605432,8,1786.718829,Kempegowda Ward,"POLYGON ((77.60961 13.11810, 77.60969 13.12308...","Kempegowda, Yelahanka Zone, Bengaluru, Bangalo..."
3,8860169645fffff,8860169645fffff,1,13.090701,77.596498,8,2635.117082,Kempegowda Ward,"POLYGON ((77.60067 13.08820, 77.60076 13.09318...","Bellary Road, Amruthnagar, Byatarayanapura, Ye..."
4,886016962dfffff,886016962dfffff,1,13.120644,77.588573,8,1853.947643,Kempegowda Ward,"POLYGON ((77.59275 13.11814, 77.59283 13.12313...","Chowdeswari Ward, Yelahanka Zone, Bengaluru, B..."


In [13]:
data = df_rent[['lat', 'lon']].applymap(math.radians).to_numpy() # Convert to radians for Haversine
tree = BallTree(data, metric = 'haversine')

n = 5 # Number of nearest neighbors to use
hexagons = df_hex[['centre_lat', 'centre_lon']].applymap(math.radians).to_numpy() # Convert to radians for Haversine
neighbor_distances, neighbour_indexes = tree.query(hexagons, n)

# Calculate weighted average prices
hex_prices = []
power = 1 # Weight is calculated as 1/d^power

for distances, indexes in zip(neighbor_distances, neighbour_indexes):
    weights = np.array([1/(d**power) for d in distances])
    prices = np.array([df_rent.loc[i, 'price'] for i in indexes])
    interpolated_price = sum(weights * prices)/sum(weights)
    hex_prices.append(interpolated_price)

df_hex['cost_sqft'] = hex_prices
df_hex.head()

,id,hex_id,ward_no,centre_lat,centre_lon,resolution,pop_total,ward_name,geometry,address,cost_sqft
0,8861892db3fffff,8861892db3fffff,1,13.113109,77.609520,8,1413.220043,Kempegowda Ward,"POLYGON ((77.61369 13.11060, 77.61378 13.11559...","Yelahanka, Kempegowda, Yelahanka Zone, Bengalu...",4896.586595
1,886016975dfffff,886016975dfffff,1,13.128071,77.609773,8,1762.379434,Kempegowda Ward,"POLYGON ((77.61395 13.12557, 77.61403 13.13055...","Kempegowda, Yelahanka Zone, Bengaluru, Bangalo...",4970.421400
2,8860169759fffff,8860169759fffff,1,13.120601,77.605432,8,1786.718829,Kempegowda Ward,"POLYGON ((77.60961 13.11810, 77.60969 13.12308...","Kempegowda, Yelahanka Zone, Bengaluru, Bangalo...",4877.673619
3,8860169645fffff,8860169645fffff,1,13.090701,77.596498,8,2635.117082,Kempegowda Ward,"POLYGON ((77.60067 13.08820, 77.60076 13.09318...","Bellary Road, Amruthnagar, Byatarayanapura, Ye...",5134.151371
4,886016962dfffff,886016962dfffff,1,13.120644,77.588573,8,1853.947643,Kempegowda Ward,"POLYGON ((77.59275 13.11814, 77.59283 13.12313...","Chowdeswari Ward, Yelahanka Zone, Bengaluru, B...",4827.231325


Plot this data to a choropleth map for visualization.

In [14]:
map_centre = (12.9792,77.5916)

m = folium.Map(location = map_centre, zoom_start = 11)

folium.Choropleth(
    geo_data = '../data/BBMP_hex.geojson',
    name = 'real_estate_costs',
    data = df_hex,
    columns = ['id', 'cost_sqft'],
    key_on = 'feature.properties.id',
    fill_color = 'YlGnBu',
    fill_opacity = 0.9,
    nan_fill_opacity = 0.3,
    line_opacity = 0.9,
    legend_name = 'Real Estate Price (Rs. per sq. ft.)',
).add_to(m)

m # Display map

As expected, real estate in the central parts of the city is significantly more expensive.

In [15]:
# Save the data to file
df_hex.to_feather('../data/bangalore_hex_costs.feather')

<ipython-input-15-87448d815c0e>:2: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  df_hex.to_feather('../data/bangalore_hex_costs.feather')
